Fitting the parity function
===========================

This example shows that a variational circuit can
be optimized to emulate the parity function

$$\begin{aligned}
f: x \in \{0,1\}^{\otimes n} \rightarrow y =
\begin{cases} 1 \text{  if uneven number of 1's in } x \\ 0
\text{ else}. \end{cases}
\end{aligned}$$

We are building a ML model which will demonstrate how to encode binary inputs into the initial state of
the variational circuit, which is simply a computational basis state
(*basis encoding*).

In [1]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.1 MB/s eta 0:00:00


In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from typing import Tuple

In [3]:
# Configuration for the model and training
config = {
    "num_qubits": 4, # Number of qubits in the quantum circuit
    "num_layers": 2, # Number of layers in the variational circuit
    "learning_rate": 0.5, # Learning rate for the optimizer
    "batch_size": 5, # Batch size for training
    "num_iterations": 100, # Number of training iterations
    "train_data_path": "/content/train data.txt", # Path to the training data file
    "test_data_path": "/content/test data.txt" # Path to the testing data file
}

In [4]:
# Data loading
def load_data(file_path: str) -> Tuple[np.ndarray, np.ndarray]:
    """Loads data from a text file.

    Args:
        file_path (str): The path to the data file.

    Returns:
        tuple: A tuple containing the input features (X) and labels (Y).
    """
    data = np.loadtxt(file_path, dtype=int)
    X = np.array(data[:, :-1])
    Y = np.array(data[:, -1])
    Y = Y * 2 - 1  # Shifts labels from {0, 1} to {-1, 1} for binary classification
    return X, Y


#Define Quantum Circuit Layer

This cell defines a function `layer` that represents a single layer of the variational quantum circuit. It applies rotation gates to each qubit and CNOT gates between pairs of qubits.

In [5]:
# Quantum circuit definition
def layer(layer_weights):
    """A single layer of the variational circuit."""
    for wire in range(config["num_qubits"]):
        qml.Rot(*layer_weights[wire], wires=wire) # Applies a rotation gate to each qubit

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)

In [6]:
# State preparation function
def state_preparation(x):
    """Encodes the input data into the quantum state."""
    qml.BasisState(x, wires=range(config["num_qubits"])) # Encodes the input as a basis state

#Quantum Device

This cell initializes a quantum device dev using qml.device. The "default.qubit" device simulates a quantum computer, allowing us to run and test the quantum circuit without access to a physical quantum computer.

#Quantum Node

This cell defines the core quantum computation as a quantum node circuit using the @qml.qnode decorator.


*   **Input and Weights:** The function takes the trainable weights and the input data x as parameters.
*   **State Preparation and Layers:** It first prepares the quantum state using state_preparation(x) and then applies the layers of the variational circuit using the provided weights.


*   **Measurement:** Finally, it measures the expectation value of the Pauli Z operator (qml.PauliZ(0)) on the first qubit. This expectation value represents the output of the quantum circuit and will be used for classification.






In [7]:
# Quantum device initialization
dev = qml.device("default.qubit")
@qml.qnode(dev)
def circuit(weights, x):
    """The quantum circuit for the variational classifier."""
    state_preparation(x)

    for layer_weights in weights:
        layer(layer_weights) # Applies each layer to the quantum state

    return qml.expval(qml.PauliZ(0))

#Variational Classifier Model

This cell defines the variational_classifier function, which represents the overall model. It takes the weights, bias, and input x as parameters. It computes the output of the quantum circuit circuit(weights, x) and adds a bias term to it. This output serves as the model's prediction.

In [8]:
def variational_classifier(weights, bias, x):
    """The variational classifier model."""
    return circuit(weights, x) + bias


#Loss and Accuracy Functions

This cell defines two helper functions:

*   **square_loss:** Calculates the mean squared error (MSE) loss between the true labels and the model's predictions. MSE is a common loss function for regression tasks.
*   **accuracy:** Calculates the accuracy of the model's predictions by counting the number of correct predictions and dividing by the total number of predictions.



In [9]:
# Loss and accuracy functions
def square_loss(labels, predictions):
    """Calculates the mean squared error loss."""
    return np.mean((labels - qml.math.stack(predictions)) ** 2)

def accuracy(labels, predictions):
    """Calculates the accuracy of the predictions."""
    acc = sum(abs(l - p) < 1e-5 for l, p in zip(labels, predictions))
    return acc / len(labels)


#Cost Function

This cell defines the cost function, which is the objective function that the optimizer will try to minimize during training. It takes the weights, bias, input features `X`, and true labels `Y` as parameters.

**Prediction Calculation:** It first computes the predictions for all inputs using the `variational_classifier`.

**Loss Calculation:** It then calculates the mean squared error loss between the true labels and the predictions.

In [10]:
# Cost function
def cost(weights, bias, X, Y):
    """The cost function to be minimized."""
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions) # Returns the mean squared error loss


#Model Training

This cell defines the train_model function that orchestrates the training process.

**Initialization:** It initializes the weights randomly and the bias to zero. The `requires_grad=True` flag indicates that these parameters should be tracked for gradient computation during optimization.

**Training Loop:** It iterates over a specified number of training iterations. In each iteration:


*   It randomly selects a batch of data.
*   It updates the weights and bias using the optimizer's `step` method, which computes gradients and adjusts the parameters to minimize the cost function.

*   It calculates and prints the current cost and accuracy on the entire training set to monitor progress.

**Return Trained Parameters:** After training, it returns the optimized weights and bias.

In [11]:
# Model training
def train_model(X, Y, optimizer, batch_size, num_iterations):
    """Trains the variational classifier."""
    np.random.seed(0)
    weights_init = 0.01 * np.random.randn(config["num_layers"], config["num_qubits"], 3, requires_grad=True)
    bias_init = np.array(0.0, requires_grad=True)

    weights = weights_init
    bias = bias_init

    for it in range(num_iterations):
        batch_index = np.random.randint(0, len(X), (batch_size,))
        X_batch = X[batch_index] # Extracts input features for the batch
        Y_batch = Y[batch_index] # Extracts labels for the batch
        weights, bias = optimizer.step(cost, weights, bias, X=X_batch, Y=Y_batch) # Updates weights and bias using the optimizer

        # Compute and print training progress
        predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
        current_cost = cost(weights, bias, X, Y)
        acc = accuracy(Y, predictions)
        print(f"Iter: {it+1:4d} | Cost: {current_cost:0.7f} | Accuracy: {acc:0.7f}")

    return weights, bias


#Main Execution (Training)

This cell is the main execution block for training the model. It checks if the script is being run as the main program (not imported as a module). If so, it:

**Loads Data:** Loads the training and testing data using the `load_data` function.

**Initializes Optimizer:** Creates an instance of the Nesterov Momentum Optimizer with the specified learning rate.

**Trains Model:** Calls the `train_model` function to train the variational classifier using the training data and the optimizer.

In [12]:
# Main execution
if __name__ == "__main__":
    X_train, Y_train = load_data(config["train_data_path"])
    X_test, Y_test = load_data(config["test_data_path"])

    opt = NesterovMomentumOptimizer(config["learning_rate"])
    weights, bias = train_model(X_train, Y_train, opt, config["batch_size"], config["num_iterations"])

Iter:    1 | Cost: 2.3147651 | Accuracy: 0.5000000
Iter:    2 | Cost: 1.9664866 | Accuracy: 0.5000000
Iter:    3 | Cost: 1.9208589 | Accuracy: 0.5000000
Iter:    4 | Cost: 2.6276126 | Accuracy: 0.5000000
Iter:    5 | Cost: 0.9323119 | Accuracy: 0.6000000
Iter:    6 | Cost: 1.1903549 | Accuracy: 0.5000000
Iter:    7 | Cost: 2.0508989 | Accuracy: 0.4000000
Iter:    8 | Cost: 1.1275531 | Accuracy: 0.6000000
Iter:    9 | Cost: 1.1659803 | Accuracy: 0.6000000
Iter:   10 | Cost: 1.1349618 | Accuracy: 0.6000000
Iter:   11 | Cost: 0.9994063 | Accuracy: 0.6000000
Iter:   12 | Cost: 1.0812559 | Accuracy: 0.6000000
Iter:   13 | Cost: 1.2863155 | Accuracy: 0.6000000
Iter:   15 | Cost: 1.1323724 | Accuracy: 0.6000000
Iter:   16 | Cost: 1.3439737 | Accuracy: 0.8000000
Iter:   17 | Cost: 2.0076168 | Accuracy: 0.6000000
Iter:   18 | Cost: 1.2685760 | Accuracy: 0.5000000
Iter:   19 | Cost: 1.6762475 | Accuracy: 0.5000000
Iter:   20 | Cost: 1.1868237 | Accuracy: 0.6000000
Iter:   21 | Cost: 1.4784687 | 

#Model Evaluation

This cell defines the evaluate_model function, which evaluates the performance of the trained model on the test set.

**Prediction Calculation:** It computes predictions for all test inputs using the trained weights and bias.
**Individual Predictions:** It iterates through the test inputs, true labels, and predictions, printing each individual prediction.
**Accuracy Calculation:** It calculates and prints the overall accuracy of the model on the test set.

In [13]:
# Model evaluation
def evaluate_model(weights, bias, X, Y):
    """Evaluates the trained model on the test set."""
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]

    for x, y, p in zip(X, Y, predictions):
        print(f"x = {x}, y = {y}, pred={p}")

    acc = accuracy(Y, predictions)
    print("Accuracy on unseen data:", acc)

#Main Execution (Evaluation)

This cell is the main execution block for evaluating the trained model. It checks if the script is being run as the main program. If so, it:


*   **Loads Testing Data:** Loads the testing data using the `load_data` function.
*   **Evaluates Model:** Calls the `evaluate_model` function to evaluate the trained model on the testing data, providing insights into the model's generalization performance on unseen data.



In [14]:
# Main execution (evaluation part)
if __name__ == "__main__":
    X_test, Y_test = load_data(config["test_data_path"])
    evaluate_model(weights, bias, X_test, Y_test)

x = [0 0 0 0], y = -1, pred=-1.0
x = [0 0 1 1], y = -1, pred=-1.0
x = [1 0 1 0], y = -1, pred=-1.0
x = [1 1 1 0], y = 1, pred=1.0
x = [1 1 0 0], y = -1, pred=-1.0
x = [1 1 0 1], y = 1, pred=1.0
Accuracy on unseen data: 1.0


This comprehensive explanation covers the purpose and functionality of each step in the provided code, offering a detailed understanding of how the variational quantum classifier is constructed, trained, and evaluated.